In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-pac

### Loading article df

In [2]:
data_folder = "../../../data/"
import json
with open(data_folder + 'insight_db.articles.json') as f:
    insight_articles = json.load(f)

In [3]:
valid_articles = {}

for k in insight_articles:
    if 'is_premium_article' in k and k['is_premium_article'] == False:
        article_id = k['_id']['$oid']
        valid_articles[article_id] = k
        valid_articles[article_id]['source_id'] = k['source_id']['$oid']

In [4]:
len(valid_articles)

172945

In [5]:
import numpy as np
import pandas as pd

In [8]:
all_articles_df = pd.DataFrame(valid_articles).T.reset_index()
all_articles_df = all_articles_df.rename(columns = {'index': 'article_id'})
all_articles_df = all_articles_df.drop(columns = {'_id', '_class'})

In [9]:
all_articles_df[all_articles_df.groupby('url')['url'].transform('count') > 1].sort_values('url').to_csv('duplicate_articles_df.csv')

In [6]:
articles_df = pd.DataFrame(valid_articles).T.reset_index()
articles_df = articles_df.rename(columns = {'index': 'article_id'})
articles_df = articles_df.drop(columns = {'_id', '_class'})
articles_df['prob'] = [np.random.rand() for i in range(len(articles_df))]

In [7]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [8]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT * FROM source_id_characteristics'
    cursor.execute(sql)
    results = cursor.fetchall()

In [9]:
results

[(4, '652d53256a2736f06f46cfcf', Decimal('1'), 'wint-wealth'),
 (3, '65291b1e9a2fbc229e5f29c9', Decimal('1'), 'zerodha-varsity'),
 (5, '6530d40d9a7559d3ec6b9871', Decimal('1.0'), 'groww'),
 (1, '6512cdcad01a9c8e86263e05', Decimal('0.9'), 'economic-times'),
 (2, '650046bd005149c49201269f', Decimal('0.9'), 'live-mint'),
 (10, '65558d1f672118037c541088', Decimal('1.0'), 'navi'),
 (11, '655597ea96fb32eef21a562a', Decimal('0.9'), 'investopedia'),
 (12, '6555987996fb32eef21a562b', Decimal('0.9'), 'business-standard'),
 (13, '655598ab96fb32eef21a562c', Decimal('0.9'), 'business-insider'),
 (14, '6555992096fb32eef21a562d', Decimal('0.9'), 'cnbc-tv-18'),
 (15, '6555995596fb32eef21a562e', Decimal('0.9'), 'cnbc'),
 (16, '65559d0c96fb32eef21a562f', Decimal('0.9'), 'business-world'),
 (17, '65559d4296fb32eef21a5630', Decimal('0.9'), 'bq-prime'),
 (18, '6560a5bddf69601d3e2ce0c1', Decimal('0.9'), 'bbc'),
 (19, '6560ac8b9952a16bbc397223', Decimal('0.9'), 'yahoo-finance'),
 (20, '6560af039952a16bbc3972

In [10]:
source_df = pd.DataFrame(results, columns = ['id', 'source_id', 'decay', 'source_name'])

In [11]:
articles_df = pd.merge(articles_df, source_df, how = 'left', on = 'source_id')

### Finding any removing duplicate articles

In [ ]:
articles_df

,article_id,url,source_id,authors,reactions,image_url,is_premium_article,last_updated_time,published_time,short_description,tags,title,ai_generated_info,prob,id,decay,source_name
0,658d3bf54b25ce4814fb2fc9,https://www.moneycontrol.com/news/business/eco...,6560af039952a16bbc397224,[Faizan Javed],{},https://images.moneycontrol.com/static-mcnews/...,False,2023-12-28T05:08:01+0000,2023-12-28T04:52:37+0000,The revised model articles of associations/bye...,"[Economy,India,Commerce Ministry,EPC,FIEO]",Commerce ministry proposes new bye-laws for el...,{'summary': {'OpenHermes_WatermelonSapphireZip...,0.407601,20,0.9,money-control
1,658d3bf54b25ce4814fb2fca,https://www.moneycontrol.com/news/business/6-r...,6560af039952a16bbc397224,[101Reporters],{},https://images.moneycontrol.com/static-mcnews/...,False,2023-12-27T13:04:27+0000,2023-12-27T13:04:27+0000,Six relatives of Andhra Pradesh MLA died in a ...,[Car accident],6 relatives of Andhra Pradesh MLA die in car a...,{'summary': {'OpenHermes_WatermelonSapphireZip...,0.590986,20,0.9,money-control
2,658d3bf64b25ce4814fb2fcb,https://www.moneycontrol.com/news/business/eco...,6560af039952a16bbc397224,[Faizan Javed],{},https://images.moneycontrol.com/static-mcnews/...,False,2023-12-28T08:08:34+0000,2023-12-28T06:37:37+0000,"From floods to droughts, extreme weather event...","[Economy,India,Inflation,food]",Govt hopeful of 'bountiful harvest' amid food ...,{'summary': {'OpenHermes_WatermelonSapphireZip...,0.547820,20,0.9,money-control
3,658d3bf64b25ce4814fb2fcd,https://www.moneycontrol.com/news/business/mor...,6560af039952a16bbc397224,[Shantanu Bhattacharji],{},https://images.moneycontrol.com/static-mcnews/...,False,2023-12-28T02:19:03+0000,2023-12-28T02:19:03+0000,A round-up of top newspaper stories to keep yo...,"[BSE, NSE, Go First, Sony, ZEE, Wipro, Cogniza...",Morning Scan: All the big stories to get you s...,{'summary': {'OpenHermes_WatermelonSapphireZip...,0.350942,20,0.9,money-control
4,658d3bf84b25ce4814fb2fce,https://www.moneycontrol.com/news/business/ive...,6560af039952a16bbc397224,[M Snehanjali],{},https://images.moneycontrol.com/static-mcnews/...,False,2023-12-27T15:29:32+0000,2023-12-27T15:29:32+0000,The hearing-aid tech will be licensed to any g...,[Dave Lee],"I’ve had trouble hearing, then I tried new gla...",{'summary': {'OpenHermes_WatermelonSapphireZip...,0.294836,20,0.9,money-control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172940,658ced9e4b25ce4814fb2a0d,https://www.cnbc.com/2023/06/16/south-korea-re...,6555995596fb32eef21a562e,[],{},https://image.cnbcfm.com/api/v1/image/10725799...,False,2023-06-16T05:23:42+0000,2023-06-16T05:23:42+0000,"The debris was salvaged on Thursday, the milit...","[Defense,Government and politics,Foreign polic...",South Korea recovers part of rocket used in No...,{'summary': {'OpenHermes_WatermelonSapphireZip...,0.681903,15,0.9,cnbc
172941,658ceda04b25ce4814fb2a60,https://www.cnbc.com/2023/05/23/dollar-hits-6-...,6555995596fb32eef21a562e,[],{},https://image.cnbcfm.com/api/v1/image/10707652...,False,2023-05-23T20:09:01+0000,2023-05-23T05:21:00+0000,The U.S. dollar hit a two-month high against a...,"[Breaking News: Markets,Forex markets,Economic...",Dollar higher as U.S. debt ceiling concerns ke...,{'summary': {'OpenHermes_WatermelonSapphireZip...,0.712452,15,0.9,cnbc
172942,658ced994b25ce4814fb295d,https://www.cnbc.com/2023/06/06/4point9-magnit...,6555995596fb32eef21a562e,[],{},https://image.cnbcfm.com/api/v1/image/10725191...,False,2023-06-06T17:08:30+0000,2023-06-06T17:08:30+0000,An earthquake with a preliminary magnitude of ...,"[Earthquakes,Haiti,Environment,business news]",4.9 magnitude quake strikes southern Haiti; 4 ...,{'summary': {'OpenHermes_WatermelonSapphireZip...,0.133102,15,0.9,cnbc
172943,658ced9e4b25ce4814fb2a00,https://www.cnbc.com/2023/12/02/north-korea-sa...,6555995596fb32eef21a562e,[],{},https://image.cnbcfm.com/api/v1/image/10734224...,False,2023-12-02T08:27:02+0000,2023-12-02T08:27:02+0000,North K

In [12]:
round(100 * (articles_df.groupby('source_name')['url'].count() - articles_df.groupby('source_name')['url'].nunique())/articles_df.groupby('source_name')['url'].count())

source_name
bbc                 0.0
bq-prime            0.0
business-insider    0.0
business-world      0.0
cnbc                0.0
cnbc-tv-18          0.0
economic-times      0.0
economist           0.0
finshots            0.0
groww               0.0
inc42               0.0
investopedia        0.0
live-mint           0.0
money-control       0.0
navi                0.0
reuters             0.0
wint-wealth         0.0
zerodha-varsity     0.0
Name: url, dtype: float64

In [14]:
articles_df.groupby('url')['url'].count().nlargest(30)

url
https://economictimes.indiatimes.com/industry/auto/auto-components/adoption-of-electric-vehicles-expected-to-accelerate-in-coming-years-rane-brake-linings-ltd/articleshow/101434376.cms                  1
https://economictimes.indiatimes.com/industry/auto/auto-components/assurance-intl-goodyear-unveil-new-range-of-filters-batteries-in-india/articleshow/102611689.cms                                       1
https://economictimes.indiatimes.com/industry/auto/auto-components/auto-component-industry-reports-highest-turnover-in-fy23-aims-to-log-double-digit-growth-this-fiscal/articleshow/102501507.cms         1
https://economictimes.indiatimes.com/industry/auto/auto-components/delphi-tvs-technologies-to-take-up-capital-expansion-at-chennai-plant/articleshow/99403761.cms                                         1
https://economictimes.indiatimes.com/industry/auto/auto-components/eveready-industries-cuts-number-of-direct-distributors-to-1000-from-5000-earlier/articleshow/100279158.cms       

In [18]:
articles_df = articles_df.drop_duplicates('url')

In [15]:
articles_df.groupby('source_name')['url'].count()

source_name
bbc                  2044
bq-prime             1117
business-insider     2872
business-world       7753
cnbc                15576
cnbc-tv-18          12635
economic-times      24911
economist             554
finshots             1760
groww                1736
inc42                 237
investopedia        20448
live-mint           27469
money-control       48339
navi                 1714
reuters              1163
wint-wealth          1743
zerodha-varsity       874
Name: url, dtype: int64

In [20]:
articles_df.groupby('url')['source_name'].nunique().nlargest()

url
https://economictimes.indiatimes.com/industry/auto/auto-components/adoption-of-electric-vehicles-expected-to-accelerate-in-coming-years-rane-brake-linings-ltd/articleshow/101434376.cms             1
https://economictimes.indiatimes.com/industry/auto/auto-components/assurance-intl-goodyear-unveil-new-range-of-filters-batteries-in-india/articleshow/102611689.cms                                  1
https://economictimes.indiatimes.com/industry/auto/auto-components/auto-component-industry-reports-highest-turnover-in-fy23-aims-to-log-double-digit-growth-this-fiscal/articleshow/102501507.cms    1
https://economictimes.indiatimes.com/industry/auto/auto-components/delphi-tvs-technologies-to-take-up-capital-expansion-at-chennai-plant/articleshow/99403761.cms                                    1
https://economictimes.indiatimes.com/industry/auto/auto-components/eveready-industries-cuts-number-of-direct-distributors-to-1000-from-5000-earlier/articleshow/100279158.cms                        1
N

In [88]:
articles_df[articles_df.title == 'A Step-by-Step Guide to UAN Generation and Activation Process'].url.values

array(['https://www.wintwealth.com/blog/a-step-by-step-guide-to-uan-generation-and-activation-process',
       'https://www.wintwealth.com/blog/a-step-by-step-guide-to-uan-generation-and-activation-process-2',
       'https://navi.com/blog/uan/'], dtype=object)

In [89]:
articles_df[articles_df.title == 'Commerce ministry asks exporters to focus on potential sectors, key markets to boost exports'].url.values

array(['https://economictimes.indiatimes.com/news/economy/foreign-trade/commerce-ministry-asks-exporters-to-focus-on-potential-sectors-key-markets-to-boost-exports/articleshow/101464696.cms',
       'https://www.cnbctv18.com/business/commerce-ministry-asks-exporters-to-focus-on-potential-sectors-key-markets-to-boost-exports-17105661.htm'],
      dtype=object)

In [96]:
articles_df[articles_df.title == 'Oil extends gains as OPEC+ to mull deeper cuts'].url.values

array(['https://www.cnbctv18.com/market/oil-extends-gains-as-opec-to-mull-deeper-cuts-18362691.htm',
       'https://www.cnbc.com/2023/11/20/oil-extends-gains-as-opec-to-mull-deeper-cuts.html'],
      dtype=object)

In [94]:
pd.options.display.max_rows = 200

In [95]:
articles_df[articles_df.groupby(['title'])['source_name'].transform('nunique') > 1].groupby('title')['source_name'].unique()

title
A Step-by-Step Guide to UAN Generation and Activation Process                                                     [wint-wealth, navi]
Adani Ports' move to buy back USD 130-million bonds an opportunistic exchange: S&P Global Ratings        [economic-times, cnbc-tv-18]
Adani Wilmar files police complaint against sale of counterfeit Fortune brand products                   [economic-times, cnbc-tv-18]
Airbnb inks pact with tourism ministry to promote heritage stays, cultural tourism                       [economic-times, cnbc-tv-18]
Amtrak train with 198 passengers derails after hitting truck on tracks in Southern California                  [cnbc, economic-times]
B'luru, H'bad, Chennai contribute 59 pc to total office leasing across 7 cities in Apr-Jun: Report       [economic-times, cnbc-tv-18]
BHEL logs over 17% rise in new orders in 2022-23                                                         [economic-times, cnbc-tv-18]
Bajaj Allianz Life declares bonus worth Rs 1,201 crore i

In [16]:
articles_df.groupby('source_name')['url'].count()/len(articles_df)

source_name
bbc                 0.011819
bq-prime            0.006459
business-insider    0.016606
business-world      0.044829
cnbc                0.090063
cnbc-tv-18          0.073058
economic-times      0.144040
economist           0.003203
finshots            0.010177
groww               0.010038
inc42               0.001370
investopedia        0.118234
live-mint           0.158831
money-control       0.279505
navi                0.009911
reuters             0.006725
wint-wealth         0.010078
zerodha-varsity     0.005054
Name: url, dtype: float64

### Extracting cat, subcat

In [68]:
articles_df[articles_df.source_name == 'bbc'].sample(5).url.values

array(['https://www.bbc.com/news/business-65022389',
       'https://www.bbc.com/news/business-65623561',
       'https://www.bbc.com/news/business-67125913',
       'https://www.bbc.com/news/business-65187441',
       'https://www.bbc.com/news/business-65272827'], dtype=object)

In [57]:
articles_df[articles_df.source_name == 'bq-prime'].sample(5).url.values

array(['https://www.bqprime.com/business/eight-indigo-passengers-left-stranded-in-bengaluru-airline-blames-lack-of-time',
       'https://www.bqprime.com/business/government-cuts-windfall-tax-on-crude-diesel',
       'https://www.bqprime.com/business/maharashtra-needs-153-trillion-investment-in-6-years-for-its-1-trillion-economy-vision',
       'https://www.bqprime.com/business/indigo-parent-interglobe-plans-to-launch-electric-air-taxi-service-in-india-in-2026',
       'https://www.bqprime.com/markets/fpis-stay-net-sellers-for-the-14th-straight-day-3'],
      dtype=object)

In [66]:
# no proper structure cat in business insider
articles_df[articles_df.source_name == 'business-insider'].sample(5).url.values

array(['https://www.businessinsider.com/sc/how-twilio-is-embracing-ai-to-personalize-engagement',
       'https://www.businessinsider.com/guides/learning/princeton-university-free-online-courses',
       'https://www.businessinsider.com/moving-to-leaving-florida-reasons-home-prices-affordability-weather-2023-11',
       'https://www.businessinsider.com/kaiser-permanente-worker-strike-union-patient-health-risk-2023-9',
       'https://www.businessinsider.com/why-apple-has-smarter-pricing-strategy-than-facebook-headset-2023-6'],
      dtype=object)

In [67]:
articles_df[articles_df.source_name == 'business-world'].sample(5).url.values

array(['https://www.bworldonline.com/world/2023/11/16/557974/czech-government-freezes-russian-states-real-estate-assets/',
       'https://www.bworldonline.com/infographics/2023/07/26/535964/philippines-ranks-12th-most-attractive-country-for-offshore-services/',
       'https://www.bworldonline.com/world/2023/07/21/535324/chinese-hackers-breach-us-ambassadors-emails-wsj/',
       'https://www.bworldonline.com/corporate/2023/04/21/518191/globe-smart-still-keen-on-longer-sim-deadline/',
       'https://www.bworldonline.com/agribusiness/2023/10/02/549016/phl-2024-broiler-chicken-production-forecast-at-1-53-million-metric-tons/'],
      dtype=object)

In [71]:
articles_df[articles_df.source_name == 'cnbc'].sample(5).url.values

array(['https://www.cnbc.com/2023/08/31/trump-pleads-not-guilty-to-election-charges-in-georgia-waives-arraignment.html',
       'https://www.cnbc.com/2023/08/24/first-trial-of-trump-co-defendant-in-georgia-case-will-begin-oct-23-judge-rules.html',
       'https://www.cnbc.com/2023/11/02/starbucks-sbux-earnings-q4-2023.html',
       'https://www.cnbc.com/2023/09/17/russell-brands-agency-terminates-ties-following-sexual-assault-allegations.html',
       'https://www.cnbc.com/2023/10/04/health-care-workers-strike-at-kaiser-permanente.html'],
      dtype=object)

In [75]:
articles_df[articles_df.source_name == 'cnbc-tv-18'].sample(5).url.values

array(['https://www.cnbctv18.com/finance/canfin-homes-ceo-exclusive-interview-suresh-srinivasan-iyer-fraud-ambala-branch-cheque-process-17333691.htm',
       'https://www.cnbctv18.com/market/nse-signs-mou-with-maharashtra-moneybee-institute-to-create-financial-literacy-nagpur-devendra-fadnavis-16941571.htm',
       'https://www.cnbctv18.com/market/stocks/tata-teleservices-new-ucaas-launch-microsoft-teams-integration-shares-rally-17275581.htm',
       'https://www.cnbctv18.com/economy/indian-govt-fta-fears-eu-carbon-tariffs-will-raise-compliance-costs-impede-free-trade-17144621.htm',
       'https://www.cnbctv18.com/market/commodities/saxo-bank-forecasts-gold-prices-to-reach-2100-dollar-per-ounce-asset-class-investment-16494651.htm'],
      dtype=object)

In [97]:
articles_df[articles_df.source_name == 'economic-times'].sample(5).url.values

array(['https://economictimes.indiatimes.com/news/india/mumbai-creek-tragedy-bodies-of-2-boys-found-1-still-missing/articleshow/101815438.cms',
       'https://economictimes.indiatimes.com/news/international/us/demon-slayer-season-4-what-you-need-to-know/articleshow/104449422.cms',
       'https://economictimes.indiatimes.com/news/india/heavy-rain-lashes-delhi-waterlogging-causes-traffic-congestion-in-parts-of-city/articleshow/101788655.cms',
       'https://economictimes.indiatimes.com/news/india/sc-trashes-appeal-of-former-isro-scientist-against-dismissal-from-service/articleshow/100232069.cms',
       'https://economictimes.indiatimes.com/news/politics-and-nation/shiv-sena-ubt-leader-sanjay-raut-backs-yogi-adityanath-on-encounters-says-no-question-of-caste-religion-if-up-cm-wants-to-end-mafia-raj/articleshow/99499420.cms'],
      dtype=object)

In [26]:
articles_df[articles_df.source_name == 'economist'].sample(5).url.values

array(['https://www.economist.com/finance-and-economics/2023/08/07/elon-musks-plans-could-hinder-twitternomics',
       'https://www.economist.com/finance-and-economics/2023/09/13/has-the-european-central-bank-become-too-powerful',
       'https://www.economist.com/business/2023/06/29/meet-the-worlds-most-flirtatious-sovereign-wealth-fund',
       'https://www.economist.com/finance-and-economics/2023/09/21/why-uranium-prices-are-soaring',
       'https://www.economist.com/economic-and-financial-indicators/2023/09/14/economic-data-commodities-and-markets'],
      dtype=object)

In [33]:
articles_df[articles_df.source_name == 'finshots'].sample(5).url.values

array(['https://finshots.in/markets/ideaforge-is-ready-to-fly-drones-over-dalal-street/',
       'https://finshots.in/archive/can-we-fix-csr/',
       'https://finshots.in/archive/can-maruti-suzuki-create-an-ev-storm-2/',
       'https://finshots.in/archive/biodiversity-act-amendments/',
       'https://finshots.in/archive/its-all-gas-2/'], dtype=object)

In [98]:
articles_df[articles_df.source_name == 'groww'].sample(5).url.values

array(['https://groww.in/blog/best-fund-managers-in-india',
       'https://groww.in/blog/apollo-hospitals-q1-results-fy22-2023-pat-down-by-35-to-rs-317-crore',
       'https://groww.in/blog/bandhan-bank-quarterly-results-q3-fy22',
       'https://groww.in/blog/best-dividend-paying-penny-stocks-in-india',
       'https://groww.in/blog/reasons-not-invest-small-cap-funds'],
      dtype=object)

In [39]:
articles_df[articles_df.source_name == 'inc42'].sample(5).url.values

array(['https://inc42.com/features/with-zero-mega-deals-in-sight-ecommerce-funding-falls-47-yoy-to-224-mn-in-q3-2023/',
       'https://inc42.com/buzz/firstcry-drhp-heres-how-the-ecommerce-unicorn-intends-to-spend-ipo-proceeds/',
       'https://inc42.com/buzz/paisabazaar-under-income-tax-depts-scanner-over-certain-vendors/',
       'https://inc42.com/resources/security-as-a-lens-for-navigating-the-future-of-digital-payments/',
       'https://inc42.com/buzz/former-unacademy-executive-vivek-sinha-nets-11-mn-for-his-new-edtech-venture/'],
      dtype=object)

In [107]:
articles_df[articles_df.source_name == 'investopedia'].sample(5).url.values

array(['https://www.investopedia.com/terms/a/at-par.asp',
       'https://www.investopedia.com/articles/financial-advisors/101315/are-etfs-good-fit-401k-plans.asp',
       'https://www.investopedia.com/mortgage-rates-continue-upward-surge-7557401',
       'https://www.investopedia.com/articles/exchangetradedfunds/11/benefits-etfs-hedging.asp',
       'https://www.investopedia.com/articles/personal-finance/120715/what-happens-retirement-accounts-if-spouse-dies.asp'],
      dtype=object)

In [101]:
articles_df[articles_df.source_name == 'live-mint'].sample(5).url.values

array(['https://www.livemint.com/market/live-blog/tata-chemicals-share-price-live-blog-for-16-nov-2023-11700102665178.html',
       'https://www.livemint.com/companies/elon-musk-s-neuralink-gets-fda-approval-for-study-of-brain-implants-in-humans-11685119212072.html',
       'https://www.livemint.com/companies/start-ups/kotak-investment-advisors-to-close-its-startup-fund-cites-falling-valuations-as-a-reason-11681394105602.html',
       'https://www.livemint.com/market/live-blog/indian-railway-catering-tourism-corporation-share-price-live-blog-for-25-oct-2023-11698202356607.html',
       'https://www.livemint.com/industry/media/adipurush-box-office-collection-movie-continues-to-mint-money-despite-controversy-11687157962872.html'],
      dtype=object)

In [59]:
articles_df[articles_df.source_name == 'money-control'].sample(5).url.values

array(['https://www.moneycontrol.com/news/business/earnings/mudra-financial-standalone-september-2023-net-sales-at-rs-0-23-crore-up-7-17-y-o-y-11795871.html',
       'https://www.moneycontrol.com/news/business/stocks/phoenix-mills-gets-overweight-rating-from-morgan-stanley-stock-gains-4-11513681.html',
       'https://www.moneycontrol.com/news/business/earnings/mayur-uniquoter-consolidated-june-2023-net-sales-at-rs-200-94-crore-up-0-25-y-o-y-11130821.html',
       'https://www.moneycontrol.com/news/business/markets/nomura-raises-tp-for-samil-by-15-following-yachiyos-4w-business-buy-10918301.html',
       'https://www.moneycontrol.com/news/business/real-estate/mhada-lottery-2023-close-to-1-lakh-applications-for-4000-plus-homes-in-mumbai-10893511.html'],
      dtype=object)

In [102]:
articles_df[articles_df.source_name == 'navi'].sample(5).url.values

array(['https://navi.com/blog/hybrid-mutual-funds/',
       'https://navi.com/blog/tpa-third-party-administrator/',
       'https://navi.com/blog/telangana-land-records/',
       'https://navi.com/blog/yield-curve-inversion/',
       'https://navi.com/blog/equity-portfoliomanagement/'], dtype=object)

In [103]:
articles_df[articles_df.source_name == 'wint-wealth'].sample(5).url.values

array(['https://www.wintwealth.com/blog/munger-commercial-real-estate-investment',
       'https://www.wintwealth.com/blog/commercial-real-estate-investment-in-muzaffarpur',
       'https://www.wintwealth.com/blog/face-value-of-shares-meaning-working-importance',
       'https://www.wintwealth.com/blog/know-the-features-and-benefits-of-a-trading-account',
       'https://www.wintwealth.com/blog/get-sukanya-samriddhi-account-passbook-online'],
      dtype=object)

In [104]:
articles_df[articles_df.source_name == 'zerodha-varsity'].sample(5).url.values

array(['https://zerodha.com/varsity/the-history-of-indian-venture-capital/',
       'https://zerodha.com/varsity/chapter/dont-deliberate-just-do-it/',
       'https://zerodha.com/varsity/chapter/getting-ready-to-trade/',
       'https://zerodha.com/varsity/chapter/disappointment-and-regret-the-other-trading-emotions/',
       'https://zerodha.com/varsity/chapter/discipline-takes-practice/'],
      dtype=object)

In [40]:
from src.articles.ArticleService import ArticleService

In [41]:
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [42]:
import re

In [70]:
ignore_first_level = ['money-control']
subcat_sources = ['live-mint','economic-times', 'cnbc-tv-18']
cat_sources = ['navi', 'zerodha-varsity', 'wint-wealth', 'investopedia', 'groww', 'business-world', 'bq-prime']

def extract_category_subcategory(row):
    # 20 and 25 identified based on iterative analysis
    source = row['source_name']
    x = row['url']
    if source in cat_sources:
        cat_url = re.split('.com/|.in/', x)[1]
        cat = cat_url.split('/')[0]
        if len(cat) < 20:
            return cat, ''
        else:
            return '',''
    elif source in subcat_sources:
        cat_url = re.split('.com/|.in/', x)[1]
        cat = cat_url.split('/')[0]
        sub_cat = cat_url.split('/')[1]
        if len(sub_cat) < 25:
            return cat, sub_cat
        else:
            return cat, ''
    elif source in ignore_first_level:
        cat_url = re.split('.com/|.in/', x)[1]
        cat_split = cat_url.split('/')
        cat = cat_split[1]
        if len(cat_split) > 2 and len(cat_split[2]) < 25:
            sub_cat = cat_split[2]
            return cat, sub_cat
        else:
            return cat, ''
    else:
        return '', ''

In [71]:
articles_df[['cat', 'subcat']] = articles_df.apply(extract_category_subcategory, axis = 1, result_type='expand')

In [72]:
articles_df.groupby('cat')['cat'].count().nsmallest()

cat
1099-g-form-4782667    1
after-8347322          1
ajay-banga-7494796     1
alphanomics-7974094    1
ameribor-6260927       1
Name: cat, dtype: int64

In [73]:
# removing those entries where there is only 1 cat count or 1 subcat count
invalid_cats = articles_df[articles_df.groupby('cat')['cat'].transform('count') == 1].cat.unique()

invalid_subcats = articles_df[articles_df.groupby('subcat')['subcat'].transform('count') == 1].subcat.unique()

articles_df.loc[articles_df.cat.isin(invalid_cats),'cat'] = ''
articles_df.loc[articles_df.subcat.isin(invalid_subcats),'subcat'] = ''

In [74]:
articles_df['fullcat'] = articles_df['source_name'] + '-' + articles_df['cat'] + '-' + articles_df['subcat']

In [75]:
len(articles_df.groupby('fullcat')['article_id'].count())

223

### Re-balancing categories for more even representation

In [76]:
fullcat_prob = articles_df.groupby('fullcat')['article_id'].count()/len(articles_df)

In [77]:
fullcat_prob.nlargest()

fullcat
money-control-business-earnings    0.099182
cnbc--                             0.090063
money-control-business-            0.065003
investopedia--                     0.054503
live-mint-market-live-blog         0.045726
Name: article_id, dtype: float64

In [448]:
reweighted_fullcat_prob = np.power(fullcat_prob, -0.5)/np.sum(np.power(fullcat_prob, -0.5))

In [449]:
fullcat_prob

fullcat
bbc--                          0.011819
bq-prime--                     0.000012
bq-prime-business-             0.004504
bq-prime-economy-finance-      0.000156
bq-prime-ipos-                 0.000358
                                 ...   
money-control-markets-bonds    0.000012
navi-blog-                     0.009911
reuters--                      0.006725
wint-wealth-blog-              0.010078
zerodha-varsity-varsity-       0.005054
Name: article_id, Length: 223, dtype: float64

In [450]:
reweighted_fullcat_prob.keys()[reweighted_fullcat_prob.argmax()]

'economic-times-industry-environment'

In [451]:
reweighted_fullcat_prob

fullcat
bbc--                          0.000432
bq-prime--                     0.013807
bq-prime-business-             0.000700
bq-prime-economy-finance-      0.003758
bq-prime-ipos-                 0.002480
                                 ...   
money-control-markets-bonds    0.013807
navi-blog-                     0.000472
reuters--                      0.000573
wint-wealth-blog-              0.000468
zerodha-varsity-varsity-       0.000660
Name: article_id, Length: 223, dtype: float64

In [452]:
reweighted_fullcat_prob['money-control-business-earnings']

0.0001490882762890219

In [453]:
reweighted_fullcat_prob.sum()

1.0

In [454]:
reweighted_fullcat_prob['cnbc--']

0.00015645360267678725

In [455]:
reweighted_fullcat_prob

fullcat
bbc--                          0.000432
bq-prime--                     0.013807
bq-prime-business-             0.000700
bq-prime-economy-finance-      0.003758
bq-prime-ipos-                 0.002480
                                 ...   
money-control-markets-bonds    0.013807
navi-blog-                     0.000472
reuters--                      0.000573
wint-wealth-blog-              0.000468
zerodha-varsity-varsity-       0.000660
Name: article_id, Length: 223, dtype: float64

In [456]:
reweighted_fullcat_prob_df = pd.DataFrame(reweighted_fullcat_prob).rename(columns = {'article_id': 'proportion'}).reset_index()

In [2]:
import numpy as np

In [6]:
type(np.random.rand())

float

In [457]:
if 'proportion' in articles_df.columns:
    articles_df = articles_df.drop(columns = {'proportion'})
articles_df = pd.merge(articles_df, reweighted_fullcat_prob_df, how = 'left')

In [458]:
baselevel = np.sum(articles_df['proportion'])

In [517]:
num_articles = 1500

In [518]:
articles_df['scaled_proportion'] = articles_df['proportion'] * (num_articles/baselevel)

In [519]:
articles_df.scaled_proportion.sum()

1500.0000000000002

In [520]:
articles_df['prob'] = [np.random.rand() for i in range(len(articles_df))]

In [521]:
articles_df['is_selected'] = articles_df['prob'] < articles_df['scaled_proportion']

In [522]:
articles_df.is_selected.sum()

1502

In [523]:
articles_df[articles_df.is_selected == 1].groupby('source_name')['is_selected'].sum().nlargest(100)

source_name
live-mint           342
money-control       311
economic-times      265
investopedia        148
cnbc-tv-18          133
business-world       97
cnbc                 41
wint-wealth          22
bq-prime             21
groww                19
business-insider     19
reuters              16
navi                 16
bbc                  15
finshots             15
economist             8
zerodha-varsity       8
inc42                 6
Name: is_selected, dtype: int64

In [530]:
shortlisted_articles = articles_df[articles_df.is_selected == True]
shortlisted_articles = shortlisted_articles.sample(frac=1)

In [524]:
representation_cutoff = int(0.05 * num_articles)

In [525]:
representation_cutoff

75

In [526]:
under_represented_sources = articles_df[articles_df.is_selected == 1].groupby('source_name').filter(lambda x: len(x) < representation_cutoff)['source_name'].unique()

In [527]:
cur_representation = articles_df[articles_df.is_selected == 1].groupby('source_name').filter(lambda x: len(x) < representation_cutoff).groupby('source_name')['source_name'].count()

In [528]:
oversampling_requirement = representation_cutoff - cur_representation

In [529]:
oversampling_requirement

source_name
bbc                 60
bq-prime            54
business-insider    56
cnbc                34
economist           67
finshots            60
groww               56
inc42               69
navi                59
reuters             59
wint-wealth         53
zerodha-varsity     67
Name: source_name, dtype: int64

In [531]:
# Empty DataFrame to hold the sampled data.
oversampled_articles_df = pd.DataFrame()

# Loop through each group, sample, and append to the sampled_df.
for group, num_samples in dict(oversampling_requirement).items():
    group_sample = articles_df[articles_df['source_name'] == group].sample(n=min(len(articles_df[articles_df['source_name'] == group]), num_samples))
    oversampled_articles_df = pd.concat([oversampled_articles_df, group_sample])

# sampled_df now contains the sampled rows.

In [532]:
oversampled_articles_df

,article_id,url,source_id,authors,reactions,image_url,is_premium_article,last_updated_time,published_time,short_description,...,prob,id,decay,source_name,cat,subcat,fullcat,scaled_proportion,is_selected,proportion
1033,658d47274b25ce4814fb361c,https://www.bbc.com/news/business-65457098,6560a5bddf69601d3e2ce0c1,[https://www.facebook.com/bbcnews],{},https://ichef.bbci.co.uk/news/1024/branded_new...,False,2023-05-02T17:38:37+0000,2023-05-02T17:38:37+0000,The owner of the dating app follows other glob...,...,0.094097,18,0.9,bbc,,,bbc--,0.008711,False,0.000432
1011,658d47494b25ce4814fb389b,https://www.bbc.com/news/business-66854695,6560a5bddf69601d3e2ce0c1,[https://www.facebook.com/bbcnews],{},https://ichef.bbci.co.uk/news/1024/branded_new...,False,2023-09-26T23:06:58+0000,2023-09-26T23:06:58+0000,"Co-founder of Little Moons, Vivien Wong, on wh...",...,0.113490,18,0.9,bbc,,,bbc--,0.008711,False,0.000432
2517,658d475c4b25ce4814fb3a00,https://www.bbc.com/news/business-66717609,6560a5bddf69601d3e2ce0c1,[https://www.facebook.com/bbcnews],{},https://ichef.bbci.co.uk/news/1024/branded_new...,False,2023-09-06T10:31:24+0000,2023-09-06T10:31:24+0000,"Some had to skip meals, take on debt or dip in...",...,0.303434,18,0.9,bbc,,,bbc--,0.008711,False,0.000432
1105,658d47264b25ce4814fb35ff,https://www.bbc.com/news/business-63486782,6560a5bddf69601d3e2ce0c1,[https://www.facebook.com/bbcnews],{},https://ichef.bbci.co.uk/news/1024/branded_new...,False,2023-08-10T12:44:55+0000,2023-08-10T12:44:55+0000,Many homeowners are worried about higher mortg...,...,0.156812,18,0.9,bbc,,,bbc--,0.008711,False,0.000432
1080,658d47534b25ce4814fb394f,https://www.bbc.com/news/business-64629081,6560a5bddf69601d3e2ce0c1,[https://www.facebook.com/bbcnews],{},https://ichef.bbci.co.uk/news/1024/branded_new...,False,2023-02-14T13:27:57+0000,2023-02-14T13:27:57+0000,Lenders will have to do better affordability c...,...,0.941664,18,0.9,bbc,,,bbc--,0.008711,False,0.000432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170769,652ebba21e5cc42b1b13995e,https://zerodha.com/varsity/chapter/working-th...,65291b1e9a2fbc229e5f29c9,[],{},https://zerodha.com/varsity/wp-content/uploads...,False,2020-01-29T06:37:15+0000,2020-01-29T06:37:15+0000,Don't let guilt paralyze you. Work through you...,...,0.383380,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.013321,False,0.000660
168757,652ebbc41e5cc42b1b139a55,https://zerodha.com/varsity/chapter/developing...,65291b1e9a2fbc229e5f29c9,[],{},https://zerodha.com/varsity/wp-content/uploads...,False,2020-05-06T06:29:46+0000,2020-05-06T06:29:46+0000,A trader's mindset plays a pivotal role in ach...,...,0.920339,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.013321,False,0.000660
169040,652ebbbf1e5cc42b1b139a1d,https://zerodha.com/varsity/chapter/overconfid...,65291b1e9a2fbc229e5f29c9,[],{},https://zerodha.com/varsity/wp-content/uploads...,False,2020-07-20T10:57:32+0000,2020-07-20T10:57:32+0000,Question your knowledge. Gain Information. Doi...,...,0.657339,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.013321,False,0.000660
169361,652ebbd11e5cc42b1b139ade,https://zerodha.com/varsity/chapter/shifting-g...,65291b1e9a2fbc229e5f29c9,[],{},https://zerodha.com/varsity/wp-content/uploads...,False,2020-12-15T06:31:41+0000,2020-12-15T06:31:41+0000,"So if you are stuck in a losing rut, don't be ...",...,0.459618,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.013321,False,0.000660


In [533]:
final_shortlisted_articles = pd.concat([shortlisted_articles, oversampled_articles_df])

In [537]:
final_shortlisted_articles

,article_id,url,source_id,authors,reactions,image_url,is_premium_article,last_updated_time,published_time,short_description,...,prob,id,decay,source_name,cat,subcat,fullcat,scaled_proportion,is_selected,proportion
162953,6571c2eb25cc9a343f03c5e2,https://www.moneycontrol.com/news/business/ear...,6560af039952a16bbc397224,[Abhishek Shingare],{},https://images.moneycontrol.com/static-mcnews/...,False,2023-08-14T20:20:50+05:30,2023-08-14T20:00:50+05:30,NaN,...,0.001560,20,0.9,money-control,business,earnings,money-control-business-earnings,0.003007,True,0.000149
120299,6571c35925cc9a343f03fcee,https://www.moneycontrol.com/news/business/cor...,6560af039952a16bbc397224,[Ankit Saproo],{},https://images.moneycontrol.com/static-mcnews/...,False,2023-04-03T07:27:23+05:30,2023-04-03T07:27:23+05:30,A round-up of top newspaper stories to keep yo...,...,0.161765,20,0.9,money-control,business,corporate-action,money-control-business-corporate-action,0.278475,True,0.013807
143284,6581079d683f430d81565e1b,https://www.cnbctv18.com/real-estate/ashiana-h...,6555992096fb32eef21a562d,[Sonal Bhutra],{},https://images.cnbctv18.com/wp-content/uploads...,False,2023-12-18T19:38:36+05:30,2023-12-18T19:38:36+05:30,"In an interview with CNBC-TV18, Varun Gupta, D...",...,0.030506,14,0.9,cnbc-tv-18,real-estate,,cnbc-tv-18-real-estate-,0.031839,True,0.001579
100815,651e02b8a662d76276b852f2,https://economictimes.indiatimes.com/news/poli...,6512cdcad01a9c8e86263e05,[PTI],{},https://economictimes.indiatimes.com/thumb/msi...,False,2023-07-03T19:08:00+05:30,2023-07-03T19:06:00+05:30,"BJP's Mahesh Jethmalani, however, made a stron...",...,0.004322,1,0.9,economic-times,news,politics-and-nation,economic-times-news-politics-and-nation,0.007129,True,0.000353
155746,6555cbee4b13023f934a30ec,https://www.cnbctv18.com/photos/market/shree-c...,6555992096fb32eef21a562d,[Kanishka Sarkar],{},https://images.cnbctv18.com/wp-content/uploads...,False,2023-05-23T09:20:32+05:30,2023-05-23T09:20:32+05:30,Brokerage calls for May 23: Macquarie has give...,...,0.015341,14,0.9,cnbc-tv-18,photos,market,cnbc-tv-18-photos-market,0.019866,True,0.000985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170769,652ebba21e5cc42b1b13995e,https://zerodha.com/varsity/chapter/working-th...,65291b1e9a2fbc229e5f29c9,[],{},https://zerodha.com/varsity/wp-content/uploads...,False,2020-01-29T06:37:15+0000,2020-01-29T06:37:15+0000,Don't let guilt paralyze you. Work through you...,...,0.383380,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.013321,False,0.000660
168757,652ebbc41e5cc42b1b139a55,https://zerodha.com/varsity/chapter/developing...,65291b1e9a2fbc229e5f29c9,[],{},https://zerodha.com/varsity/wp-content/uploads...,False,2020-05-06T06:29:46+0000,2020-05-06T06:29:46+0000,A trader's mindset plays a pivotal role in ach...,...,0.920339,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.013321,False,0.000660
169040,652ebbbf1e5cc42b1b139a1d,https://zerodha.com/varsity/chapter/overconfid...,65291b1e9a2fbc229e5f29c9,[],{},https://zerodha.com/varsity/wp-content/uploads...,False,2020-07-20T10:57:32+0000,2020-07-20T10:57:32+0000,Question your knowledge. Gain Information. Doi...,...,0.657339,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.013321,False,0.000660
169361,652ebbd11e5cc42b1b139ade,https://zerodha.com/varsity/chapter/shifting-g...,65291b1e9a2fbc229e5f29c9,[],{},https://zerodha.com/varsity/wp-content/uploads...,False,2020-12-15T06:31:41+0000,2020-12-15T06:31:41+0000,"So if you are stuck in a losing rut, don't be ...",...,0.459618,3,1,zerodha-varsity,varsity,,zerodha-varsity-varsity-,0.013321,False,0.000660


In [539]:
final_shortlisted_articles = final_shortlisted_articles.drop_duplicates('article_id')

In [540]:
final_shortlisted_articles.groupby('source_name')['url'].count()

source_name
bbc                  74
bq-prime             75
business-insider     75
business-world       97
cnbc                 75
cnbc-tv-18          133
economic-times      265
economist            74
finshots             75
groww                75
inc42                73
investopedia        148
live-mint           342
money-control       311
navi                 75
reuters              72
wint-wealth          75
zerodha-varsity      74
Name: url, dtype: int64

In [541]:
def __generate_full_content(title: str, cleaned_text: str) -> str:
    return f'|headline|: {title} \n |content|: {cleaned_text}'

def __clean_article_content(article_content: str):
    # removing new line symbol
    # article_content = article_content.replace('\n', '')
    # article_content = article_content.replace('\t', '')
    # removing characters which are not part of alphabets, numbers, currencies, percent, dot, dash, brackets, space, slash
    article_content = re.sub(r"[^a-zA-Z0-9$€₹%\-:/()\[\]{} .]", "", article_content)
    return article_content

def get_article_text(article_id):
    article_data = ArticleService.get_article_text_from_s3(article_id)
    title = article_data['meta_data']['title']
    cleaned_text = article_data['cleaned_text']
    full_content = __generate_full_content(title, cleaned_text)
    cleaned_text = __clean_article_content(full_content)
    return cleaned_text

In [542]:
final_shortlisted_articles['full_content'] = final_shortlisted_articles['article_id'].progress_apply(lambda x: get_article_text(x))

  0%|          | 0/2188 [00:00<?, ?it/s]

In [546]:
final_shortlisted_articles.iloc[0].url

'https://www.moneycontrol.com/news/business/earnings/standard-surfac-standalone-june-2023-net-sales-at-rs-29-52-crore-down-31-42-y-o-y-11186781.html'

In [544]:
final_shortlisted_articles.iloc[0].full_content

'headline: Standard Surfac Standalone June 2023 Net Sales at Rs 29.52 crore down 31.42% Y-o-Y  content: Reported Standalone quarterly numbers for Standard Surfactants are: Net Sales at Rs 29.52 crore in June 2023 down 31.42% from Rs. 43.04 crore in June 2022. Story continues below Advertisement Remove Ad Quarterly Net Profit at Rs. 0.45 crore in June 2023 down 32.55% from Rs. 0.66 crore in June 2022. EBITDA stands at Rs. 0.92 crore in June 2023 down 31.85% from Rs. 1.35 crore in June 2022. Standard Surfac EPS has decreased to Rs. 0.54 in June 2023 from Rs. 0.93 in June 2022. Standard Surfac shares closed at 57.39 on August 14 2023 (BSE) and has given -9.26% returns over the last 6 months and -15.10% over the last 12 months.'

In [548]:
final_shortlisted_articles['num_words'] = final_shortlisted_articles['full_content'].apply(lambda x: len(x.split(' ')))

In [549]:
pd.options.display.max_rows = 200

In [550]:
shortlisted_articles = final_shortlisted_articles

In [551]:
shortlisted_articles = shortlisted_articles.sample(frac=1)

In [552]:
len(shortlisted_articles)

2188

In [579]:
shortlisted_articles.to_csv('shortlisted_articles-29-Dec-23.csv')

In [556]:
shortlisted_articles.num_words.describe()

count     2188.000000
mean       705.929159
std        633.173013
min          7.000000
25%        345.750000
50%        535.000000
75%        878.250000
max      12777.000000
Name: num_words, dtype: float64

In [207]:
test_set = shortlisted_articles[:250]

In [208]:
train_set = shortlisted_articles[250:]

### Creating the prompt

In [578]:
system_prompt = '''
You are an expert chief editor for a leading Indian business and financial content website. You evaluate critical attributes of articles to gate keep content quality. The following are 18 attributes with the format of attribute: <instruction>

1. summary:  <Summarize the following article with following instructions.

a. Readability: Follow proper grammer and maintain easy readability with no more than 1-3 points and upto a total of less than 80 words. 1-2 points for short articles and 3 points for long articles
b. Faithfulness: Don't mention any details which are not part of the article
c. Accuracy: All the numbers, dates, events should have the correct attribution. Do not, at any cost mix up numbers or dates. 
d. Recall: No critical info should be missed
e. structure: Each point should be only a single line. Prefix the point with a relevant label for that point. The points should overall capture the essence of the article. The labels should be human readable and don’t use any _ or programming symbols. 
f. format: html bullet point format with bolded labels like following. We’ll call this format as _summary_html_format_

“
<ul>
  <li><strong>label1</strong>: point1</li>
  <li><strong>label2</strong>: point2</li>
 …
</ul>
“
no preamble or postamble. Make sure that you adhear to the maximum limit of 3 points. It is very important that the format is a valid html format and has bullets, lines and bolded labels>

2. summary_critique (short text): <critique the summary above by comparing with the article and the instructions. check for any important details missed or changes needed in Readability, Faithfulness, Accuracy and Recall of the expected format. Evaluate if there are too many or too few points based on article length. Access the validity of the html format as well against _summary_html_format_>
3. improved_summary: <create an improved summary by considering summary and summary_critique in the same format as a valid _summary_html_format_>
4. top_categories (5 semi colon separated words): <List 5 progressively general categories separated by semicolons (;), avoiding quotes to prevent json.loads() failures>
5. business_or_financial_article (True/False) : <True or False, based on the article's relevance to Indian corporations, investors, and policies impacting them.>
6. indian_or_international (indian/international): <is the article specific for india or is an international content>
7. relevant_for_indians (True/False) : <True or False, considering the article's relevance and applicability to Indian readers, including international and multinational contexts.>
8. article_validity_duration_evaluation (short text) : <analyse the factors for the relevance duration. stock fluctuations for 1 day; significant policy changes - few days; educational are timeless unless it references any regulations or acts, in which case a max of 30 days (regulations change). quarterly results valid for 3 days, yearly results for a 7>
9. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute among one of 1,3,7,14,30. -1 for timeless. Don’t provide any other number apart from these>
10. popularity_evaluation (short text): <evaluate the likely popularity of the article based on the following criteria:

a. reader_interest_score: how many people among casual business news readers are likely to be interested. Considering what score to give between 0 to 1
b. headline_effectiveness: ability of headline to engage. Considering what score to give between 0 to 1
c. event_novently: how frequently the event occurs or this type of content is available. every few hours, days, weeks or months. Considering what score to give between 0 to 1. Anything which happens only every few quarters is more novel. 
d. emotional_impact: what is the most common emotion that readers are going to go through and how many readers will likely have the emotion. Considering what score to give between 0 to 1

Be stingy in giving scores more than 0.4
>
11. popularity_evaluation_critique (short text): <critique the each of the scores by re looking at the article and seeing if there is anything which is over rated or anything which is under rated>
12. final_reader_interest_score: <0 to 1 float>
13. final_headline_effectiveness_score: <0 to 1 float>
14. final_event_novently_score: <0 to 1 float>
15. final_emotional_impact_score: <0 to 1 float>
16. improved_headline (short text) : <Write a headline based on the content of the article to grab attention by evoking curiosity or an emotional/intellectual response, but avoid a click bait headline>
17. article_type (fact/opinion/analysis/educational/sponsored) : <Determine if the article is fact, opinion, analysis, educational, or sponsored, based on content and presentation. Predictions without sufficient data to backup is opinion, with data is analysis. >
18. article_sentiment(bull/bear/NA): <sentiment of the article is bullish, bearish or NA. balanced is NA>


your response should be a json structure with all the 18 above keys without missing any key. It is very important that the response is directly readable with json.loads(). no preamble or postamble. 

|article|
'''

### Calling GPT-4

In [559]:
import openai

In [560]:
openai.api_key = ''

In [561]:
shortlisted_articles.iloc[0].article_id

'653168bc1e5cc42b1b13b724'

In [562]:
res1 = openai.ChatCompletion.create(model = "gpt-4", messages = [{'role': 'system', 'content': system_prompt}, {'role': 'user', 'content': shortlisted_articles.iloc[0].full_content}])

In [563]:
import json

In [564]:
parsed_json = json.loads(res1['choices'][0]['message']['content'])

In [565]:
parsed_json

{'summary': '<ul><li><strong>Investment Opportunity</strong>: Lower valuations in small- and mid-cap stocks could potentially offer Indian investors long-term advantages, suggests Vinit Sambre of DSP Mutual Fund.</li><li><strong>Investment Strategy</strong>: Investors aspiring to invest in these categories should be ready to maintain long-term outlooks of at least five years and be willing to diversify their portfolios.</li><li><strong>Market Outlook</strong>: Indian market is expected to be in a consolidation phase for the near term due to macroeconomic conditions and slowing growth rates for Indian corporations.</li></ul>',
 'summary_critique': 'The summary has captured the critical points from the article accurately following good readability, faithfulness, and recall. HTML format adheres to _summary_html_format_ and the number of points is appropriate relative to the length of the article.',
 'improved_summary': "<ul><li><strong>Investment Outlook</strong>: DSP Mutual Fund's Vinit 

In [266]:
response_tuple = [('65316faa1e5cc42b1b141024', res1)]

In [268]:
pd.DataFrame(response_tuple, columns = ['article_id', 'response']).to_csv('gpt-4-responses-22-nov.csv',header=None, mode='a')

In [280]:
pd.DataFrame(k,index=['i',])

,article_id,system_prompt,user_prompt,model,content,prompt_tokens,completion_tokens
i,65316faa1e5cc42b1b141024,\nYou are the chief editor for a leading India...,Sebi halves listing time to three days Mint: ...,gpt-4-0613,"{'analysis_is_financial_or_business_news': ""Th...",1370,535


In [566]:
from tqdm.notebook import tqdm

In [284]:
# gpt35_encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-16k")

In [283]:
# def find_token_limit(text, token_limit, l):
#     left = 0
#     right = len(text)

#     while left < right:
#         mid = left + (right - left) // 2
#         left_tokens = len(gpt35_encoding.encode(text[:mid]))
#         right_tokens = len(gpt35_encoding.encode(text[mid:]))

#         if left_tokens + right_tokens <= token_limit:
#             right = mid
#         else:
#             left = mid + 1

#     return left

In [567]:
def calculate_tokens(text, encoder):  # Placeholder for your actual token calculation function
    # Your implementation will go here.
    return len(encoder.encode(text))  # Example: counting characters as tokens

def find_token_limit(text,encoder, token_limit):
    # First, check if the whole text is under the token limit
    if calculate_tokens(text, encoder) <= token_limit:
        return text  # The entire text is within the limit

    def is_under_limit(index):
        # Use the provided function to calculate tokens for the substring
        return calculate_tokens(text[:index], encoder) <= token_limit

    left, right = 0, len(text)
    valid_limit = 0  # This will hold the index of the last valid token position

    # Binary search to find the token limit
    while left <= right:
        mid = (left + right) // 2  # Find the midpoint
        if is_under_limit(mid):
            # If the midpoint is under the limit, store it as a valid limit
            valid_limit = mid
            left = mid + 1  # Move the left boundary to the right
        else:
            right = mid - 1  # Move the right boundary to the left

    # Find the last space before the valid_limit to ensure we're at a word boundary
    space_index = text.rfind(' ', 0, valid_limit)
    if space_index == -1:
        # If there's no space, we've hit the start of the text
        return text[:valid_limit]  # Return up to the valid limit even if mid-word

    # Return the text up to the last word within the token limit
    return text[:space_index]

# Example usage:
text = "Your text goes here, and it could be very long."
token_limit = 100  # Replace with your actual token limit

In [569]:
import tiktoken

In [570]:
shorted_article_assessments = []

In [571]:
len(shorted_article_assessments)

0

In [572]:
len(shortlisted_articles)

2188

In [573]:
model = 'gpt-4'

In [575]:
def parse_gpt_api_response(article_id, system_prompt, user_prompt, api_response):
    # Load the response into a dictionary if it's a string
    if isinstance(api_response, str):
        api_response = json.loads(api_response)
    
    # Extract the required information from the response
    id = api_response.get('id')
    model = api_response.get('model')
    prompt_tokens = api_response['usage'].get('prompt_tokens')
    completion_tokens = api_response['usage'].get('completion_tokens')

    # The content within the choices is a string of json, so it needs to be loaded
    # replacers = {'\\\'':"'", '],':']",', ',[':',"['}
    try:
        content = json.loads(api_response['choices'][0]['message']['content'])
    except:
        print(f'trying to get fixed json for articleId {article_id}')
        improved_api_response = openai.ChatCompletion.create(model = 'gpt-4', messages = [{'role': 'system', 'content': 'fix the following json to be readable using json.loads(). simply out the new json. no preamble and no postamble'}, {'role': 'user', 'content': api_response['choices'][0]['message']['content']}])
        content = json.loads(improved_api_response['choices'][0]['message']['content'])
        # content = json.loads(api_response['choices'][0]['message']['content'].replace(replacers))
        # content = json.loads(api_response['choices'][0]['message']['content'].replace('],',']",').replace(',[',',"[').replace('\\\'',"'").replace('\\ "', '\\"'))
        # content = json.loads(api_response['choices'][0]['message']['content'])
    # Build the dictionary with the required information
    extracted_data = {
        'article_id': article_id,
        'system_prompt': system_prompt,
        'user_prompt': user_prompt,
        'model': model,
        'content': json.dumps(content),
        'prompt_tokens': prompt_tokens,
        'completion_tokens': completion_tokens
    }

    return extracted_data

In [460]:
import ast

In [576]:
from tqdm.notebook import tqdm

In [577]:
import json

In [20]:
encoder = tiktoken.encoding_for_model(model)
num_system_tokens = len(encoder.encode(system_prompt))
total_tokens = 7000
available_tokens = total_tokens - num_system_tokens
total_rows = len(shortlisted_articles)
for i in tqdm(range(718, total_rows)):
    row = shortlisted_articles.iloc[i]
    truncated_content = find_token_limit(text=row.full_content, encoder=encoder, token_limit=available_tokens)
    res_gpt = openai.ChatCompletion.create(model = model, messages = [{'role': 'system', 'content': system_prompt.replace('\n',' ')}, {'role': 'user', 'content': truncated_content}])
    parsed_res = parse_gpt_api_response(article_id=row.article_id, system_prompt=system_prompt, user_prompt=truncated_content, api_response=res_gpt)
    cur_df = pd.DataFrame(parsed_res, index = ['i',])
    cur_df.to_csv('gpt-4-responses-22-nov.csv',header=None, mode='a')
    shorted_article_assessments.append(parsed_res)

  0%|          | 0/108 [00:00<?, ?it/s]

trying to get fixed json for articleId 651e17b8a662d76276b88483
trying to get fixed json for articleId 65316ede1e5cc42b1b13e889
trying to get fixed json for articleId 6555cbe14b13023f934a283c
trying to get fixed json for articleId 65316e651e5cc42b1b13e4f2
trying to get fixed json for articleId 6555cbe74b13023f934a2c6c
trying to get fixed json for articleId 6531690b1e5cc42b1b13cb89
trying to get fixed json for articleId 6555c82b4b13023f9349150e


#### Analysis

In [24]:
all_article_responses = pd.read_csv('gpt-4-responses-22-nov.csv', header=None)

In [ ]:
all_article_responses.rename(columns = {'0'})

In [27]:
all_article_responses[1].nunique()

825

In [30]:
all_article_responses[6].sum() * 0.03 * 80/1000

4017.768

In [31]:
all_article_responses[7].sum() * 0.06 * 80/1000

2145.3984

In [32]:
invalid_response_article_ids = []

In [33]:
resp = []
for i in tqdm(range(len(all_article_responses))):
    row = all_article_responses.iloc[i]
    try:
        resp.append(json.loads(row[5]))
    except:
        invalid_response_article_ids.append(row[1])

  0%|          | 0/828 [00:00<?, ?it/s]

In [34]:
len(invalid_response_article_ids)

121

In [56]:
resp_df = pd.DataFrame(resp[:220])

In [61]:
resp_df['is_financial_or_business_news'] = resp_df['is_financial_or_business_news'].astype('int')
resp_df['relevant_for_india'] = resp_df['relevant_for_india'].astype('int')
resp_df['article_validity_duration'] = resp_df['article_validity_duration'].astype('int')

In [63]:
resp_df.is_financial_or_business_news.mean()

0.8863636363636364

In [64]:
resp_df.relevant_for_india.mean()

0.8090909090909091

In [69]:
resp_df.groupby('popularity')['popularity'].count()

popularity
breaking_news          33
moderately_popular    115
niche                  72
Name: popularity, dtype: int64

### Rerunning for invalid article responses

In [ ]:
invalid_response_article_ids

In [ ]:
shortlisted_articles

In [80]:
encoder = tiktoken.encoding_for_model(model)
num_system_tokens = len(encoder.encode(system_prompt))
total_tokens = 7000
available_tokens = total_tokens - num_system_tokens
total_rows = len(shortlisted_articles)
for i in tqdm(invalid_response_article_ids[92:]):
    row = shortlisted_articles.loc[shortlisted_articles.article_id == i].iloc[0]
    truncated_content = find_token_limit(text=row.full_content, encoder=encoder, token_limit=available_tokens)
    res_gpt = openai.ChatCompletion.create(model = model, messages = [{'role': 'system', 'content': system_prompt.replace('\n',' ')}, {'role': 'user', 'content': truncated_content}])
    parsed_res = parse_gpt_api_response(article_id=row.article_id, system_prompt=system_prompt, user_prompt=truncated_content, api_response=res_gpt)
    cur_df = pd.DataFrame(parsed_res, index = ['i',])
    cur_df.to_csv('gpt-4-responses-22-nov.csv',header=None, mode='a')
    shorted_article_assessments.append(parsed_res)

  0%|          | 0/29 [00:00<?, ?it/s]

In [500]:
df1 = pd.DataFrame(shorted_article_assessments)

In [503]:
df1['json_safe_content'] = df1['content'].apply(lambda x: json.dumps(x))

In [506]:
df1.to_csv('gpt-responses.csv')

In [510]:
df1['total_cost'] = df1['prompt_tokens'] * (0.03 * 80/1000) + df1['completion_tokens'] * (0.06 * 80/1000)

In [512]:
df1.total_cost.sum()

879.1272

In [495]:
pd.DataFrame(shorted_article_assessments).to_csv('gpt-responses.csv')

#### Standardizing responses

In [2]:
import pandas as pd
import numpy as np
import json

In [5]:
df = pd.read_csv('gpt-4-responses-22-nov.csv', header=None)

In [7]:
df = df.rename(columns = {1: 'article_id', 2: 'system_prompt', 3: 'article_content', 4: 'gpt-model', 5: 'gpt-response', 6: 'input_tokens', 7: 'output_tokens'})

In [157]:
article_wise_responses = {}

In [158]:
for i in range(len(df)):
    row = df.iloc[i]
    art_id = row['article_id']
    try:
        article_wise_responses[art_id] = {}
        article_wise_responses[art_id]['response'] = json.loads(row['gpt-response'])
        article_wise_responses[art_id]['content'] = row['article_content']
    except:
        pass

In [61]:
def correct_json_using_gpt(summary_json):
    print(f'trying to get fixed json for summary')
    improved_summary_json = openai.ChatCompletion.create(model = 'gpt-4', messages = [{'role': 'system', 'content': 'fix the following json to be readable using json.loads(). simply out the new json. no preamble and no postamble'}, {'role': 'user', 'content': summary_json}])
    content = json.loads(improved_summary_json['choices'][0]['message']['content'])
    return content

In [156]:
from tqdm.notebook import tqdm
import tqdm as td

from copy import deepcopy

In [185]:
response['summary_list']

[{'missing_entities': 'Nirmala Sitharaman; US India Business Council; Digitalization impact',
  'denser_summary': 'Union Finance Minister Nirmala Sitharaman urged American businesses to invest in India, asserting that it meets the requirements of a fair and transparent economy. The invitation came during a business roundtable organized by the US India Business Council. Sitharaman emphasized on the progress India has made in terms of digitalization and economic formalization.'},
 {'missing_entities': 'Taranjit Singh Sandhu; US and India trade; National Financial Information Registry',
  'denser_summary': "Indian Ambassador to the US, Taranjit Singh Sandhu reiterated India's commitment to engagement at the G20. He also noted that the US-India bilateral trade crossed $190 billion last year. The importance of a national financial information registry was emphasized to expand credit access."},
 {'missing_entities': 'Startup policies; Public Infrastructure; Fintech Ecosystem',
  'denser_summ

In [190]:
standardized_responses = {}
errenous_summaries = []
for art_id in td.tqdm(article_wise_responses.keys()):
    response = article_wise_responses[art_id]['response']
    expected_keys_1 = ['analysis_is_financial_or_business_news', 'is_financial_or_business_news', 'analysis_of_relevant_for_india', 'relevant_for_india',
                        'analysis_of_article_validity_duration', 'article_validity_duration', 'analysis_of_popularity', 'popularity', 'analysis_of_article_type', 'article_type', 
                        'analysis_of_article_sentiment', 'article_sentiment', 'headline_suggestion']
    expected_keys_2 = ['summary_list']
    if isinstance(response, list):
        res = {}
        for ele in response:
            res.update(ele)
        response = res
        # response = deepcopy(article_wise_responses[art_id]['response'][0])
        # try:
        #     response.update(article_wise_responses[art_id]['response'][1])
        # except:
        #     pass
        # assert sorted(set(response[0].keys()).intersection(expected_keys_1)) == sorted(expected_keys_1), f"{art_id}"
        # assert set(response[1].keys()) == expected_keys_2
        # try:
        #     summaries = json.loads(response[1]['summary_list'])
        # except:
        #     errenous_summaries.append(art_id)
        #     continue
        #     # print(f'articleId is {art_id}')
        #     # summaries = correct_json_using_gpt(summary_json = response[1]['summary_list'])
        # assert isinstance(summaries, list)
        # assert len(summaries) == 3
        # for summary in summaries:
        #     assert 'missing_entities' in summary and 'denser_summary' in summary
        # standardized_responses[art_id] = []
        # standardized_responses[art_id].append(response[1])
        # standardized_responses[art_id].append(summaries)
        # elif isinstance(response, dict):
    try:
        assert sorted(set(response.keys()).intersection(set(expected_keys_1).union(set(expected_keys_2)))) == sorted(set(expected_keys_1).union(set(expected_keys_2)))
    except:
        errenous_summaries.append(art_id)
        print(f'{art_id} error 0')
        continue
        # print(f'articleId is {art_id}')
        # summaries = correct_json_using_gpt(summary_json = response['summary_list'])
    try:
        if isinstance(response['summary_list'], str):
            summaries = json.loads(response['summary_list'])
        elif isinstance(response['summary_list'], list):
            summaries = response['summary_list']
    except:
        corrected_summary = correct_json_using_gpt(response['summary_list'])
        try:
            summaries = corrected_summary
        except:
            errenous_summaries.append(art_id)
            print(f'{art_id} error 1')
            continue
    assert isinstance(summaries, list)
    try:
        assert len(summaries) == 3
    except:
        print(f'{art_id} error 2')
        errenous_summaries.append(art_id)
        continue
    for summary in summaries:
        try:
            assert 'missing_entities' in summary and ('denser_summary' in summary or 'summary_text' in summary)
            if 'summary_text' in summary:
                summary['denser_summary'] = summary.pop('summary_text')
        except:
            print(f'{art_id} error 3')
            errenous_summaries.append(art_id)
            continue
    standardized_responses[art_id] = {}
    standardized_responses[art_id]['response'] = []
    attribute_response = {}
    for key in expected_keys_1:
        attribute_response[key] = response[key]
    standardized_responses[art_id]['response'].append(attribute_response)
    standardized_responses[art_id]['response'].append(summaries)
    standardized_responses[art_id]['content'] = article_wise_responses[art_id]['content']

  0%|          | 0/825 [00:00<?, ?it/s]

trying to get fixed json for summary


  0%|          | 3/825 [00:15<1:12:02,  5.26s/it]

trying to get fixed json for summary


  0%|          | 4/825 [00:35<2:16:53, 10.00s/it]

...

100%|█████████▉| 821/825 [1:07:31<00:27,  6.81s/it]

trying to get fixed json for summary


100%|█████████▉| 822/825 [1:07:55<00:27,  9.17s/it]

trying to get fixed json for summary


100%|██████████| 825/825 [1:08:09<00:00,  4.96s/it]


In [163]:
list(standardized_responses.keys())[-20:]

['6555ccc44b13023f934acd61',
 '651e0c73a662d76276b86a15',
 '651dd71da662d76276b7e3c2',
 '6555cb9f4b13023f9349fa71',
 '6555cba44b13023f9349fdd1',
 '6555cab84b13023f93495f7c',
 '6555c7ec4b13023f9348ee4a',
 '654cbbcadc4fa72a6c403e10',
 '6555c0084b13023f9348c97b',
 '6555ccd14b13023f934ad8ab',
 '651df9b1a662d76276b83c5d',
 '6555cbf64b13023f934a3718',
 '65316f1c1e5cc42b1b13efc1',
 '651e088ba662d76276b86119',
 '653168bd1e5cc42b1b13b78d',
 '655c5595eee55a44e0ac1e51',
 '6555c84a4b13023f934924ce',
 '651dca7ca662d76276b7c1c9',
 '651df878a662d76276b83979',
 '6555c80e4b13023f934901a6']

In [191]:
len(standardized_responses.keys())

821

In [192]:
len(errenous_summaries)

4

In [195]:
test_article_ids = list(df.iloc[:101].article_id.values)

In [196]:
len(set(test_article_ids).intersection(set(standardized_responses.keys())))

100

In [197]:
train_article_ids = [x for x in standardized_responses.keys() if x not in test_article_ids]

In [201]:
len(train_article_ids)

721

In [202]:
train_standardized_responses = {k: standardized_responses[k] for k in train_article_ids}

In [203]:
len(train_standardized_responses)

721

In [204]:
with open('train_set_full_27_nov.json', 'w') as f:
    json.dump(train_standardized_responses, f)

In [205]:
test_standardized_responses = {k: standardized_responses[k] for k in test_article_ids}

In [206]:
with open('test_set_full_27_nov.json', 'w') as f:
    json.dump(test_standardized_responses, f)